Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Pipeline for Automated Machine Learning
_**ディープラーンニングを利用したテキスト分類**_

## Contents
1. [事前準備](#1.-事前準備)
1. [自動機械学習 Automated Machine Learning](2.-自動機械学習-Automated-Machine-Learning)
1. [結果の確認](#3.-結果の確認)

## 1. 事前準備

本デモンストレーションでは、AutoML の深層学習の機能を用いてテキストデータの分類モデルを構築します。  
AutoML には Deep Neural Network が含まれており、テキストデータから **Embedding** を作成することができます。GPU サーバを利用することで **BERT** が利用されます。

深層学習の機能を利用するためには Azure Machine Learning の Enterprise Edition が必要になります。詳細は[こちら](https://docs.microsoft.com/en-us/azure/machine-learning/concept-editions#automated-training-capabilities-automl)をご確認ください。

## 1.1 Python SDK のインポート

Azure Machine Learning の Python SDK などをインポートします。

In [1]:
import logging
import os
import shutil

import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.run import Run
from azureml.widgets import RunDetails
from azureml.core.model import Model 
from azureml.train.automl import AutoMLConfig
from sklearn.datasets import fetch_20newsgroups

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (portalocker 2.0.0 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('portalocker~=1.0'), {'msal-extensions'}).


In [2]:
from azureml.automl.core.featurization import FeaturizationConfig

Azure ML Python SDK のバージョンが 1.8.0 以上になっていることを確認します。

In [3]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

You are currently using version 1.11.0 of the Azure ML SDK


## 1.2 Azure ML Workspace との接続

In [4]:
ws = Workspace.from_config()

In [5]:

# 実験名の指定
experiment_name = 'livedoor-news-classification-BERT-pipeline'

experiment = Experiment(ws, experiment_name)

output = {}
#output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Workspace Name,azureml_text
Resource Group,azureml_text
Location,japaneast
Experiment Name,livedoor-news-classification-BERT-pipeline


## 1.3 計算環境の準備

BERT を利用するための GPU の `Compute Cluster` を準備します。

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Compute Cluster の名称
amlcompute_cluster_name = "gpucluster"

# クラスターの存在確認
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    
except ComputeTargetException:
    print('指定された名称のクラスターが見つからないので新規に作成します.')
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6_V3",
                                                           max_nodes = 4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## 1.4 学習データの準備
今回は [livedoor New](https://www.rondhuit.com/download/ldcc-20140209.tar.gz) を学習データとして利用します。ニュースのカテゴリー分類のモデルを構築します。  

In [7]:
target_column_name = 'label'  # カテゴリーの列 
feature_column_name = 'text'  # ニュース記事の列
project_folder = './project'

In [8]:
train_dataset = Dataset.get_by_name(ws, "livedoor").keep_columns(["text","label"])
train_dataset.take(5).to_pandas_dataframe()

,label,text
0,dokujo-tsushin,「お願いがあるんだけど……友人代表のスピーチ、やってくれないかな？」 さてそんなとき、独女はどう対応したらいいか？ 最近だとインターネット等で検索すれば友人代表スピーチ用の例文サイトがたくさん出てくるので、それらを参考にすれば、無難なものは誰でも作成できる。しかし由利さん（33歳）はネットを参考にして作成したものの「これで本当にいいのか不安でした。一人暮らしなので聞かせて感想をいってくれる人もいないし、かといって他の友人にわざわざ聞かせるのもどうかと思うし……」ということで活用したのが、なんとインターネットの悩み相談サイトに。そこに作成したスピーチ文を掲載し「これで大丈夫か添削してください」とメッセージを送ったというのである。 「一晩で3人位の人が添削してくれましたよ。ちなみに自分以外にもそういう人はたくさんいて、その相談サイトには同じように添削をお願いする投稿がいっぱいありました」（由利さん）。ためしに教えてもらったそのサイトをみてみると、確かに「結婚式のスピーチの添削お願いします」という投稿が1000件を超えるくらいあった。めでたい結婚式の影でこんなネットコミュニティがあったとは知らなかった。 しかし「事前にお願いされるスピーチなら準備ができるしまだいいですよ。一番嫌なのは何といってもサプライズスピーチ！」と語るのは昨年だけで10万以上お祝いにかかったというお祝い貧乏独女の薫さん（35歳） 「私は基本的に人前で話すのが苦手なんですよ。だからいきなり指名されるとしどろもどろになって何もいえなくなる。そうすると自己嫌悪に陥って終わった後でもまったく楽しめなくなりますね」 サプライズスピーチのメリットとしては、準備していない状態なので、フランクな本音をしゃべってもらえるという楽しさがあるようだ。しかしそれも上手に対応できる人ならいいが、苦手な人の場合だと「フランク」ではなく「しどろもどろ」になる危険性大。ちなみにプロの司会者の場合、本当のサプライズではなく式の最中に「のちほどサプライズスピーチとしてご指名させていただきます」という一言があることも多いようだが、薫さん曰く「そんな何分前に言われても無理！」らしい。要は「サプライズを楽しめる」というタイプの人選が大切ということか。 一方「ありきたりじゃつまらないし、ネットで例文を検索している際に『こんな方法もあるのか！』って思って取り入れました」という幸恵さん（30歳）が行ったスピーチは「手紙形式のスピーチ」というもの。 「○○ちゃんへ みたいな感じで新婦の友人にお手紙を書いて読み上げるやり方です。これなら多少フランクな書き方でも大丈夫だし、何より暗記しないで堂々と読み上げることができますよね。読んだものはそのまま友人にあげれば一応記念にもなります」（幸恵さん）なるほど、確かにこれなら読みあげればいいだけなので、人前で話すのが苦手な人でも失敗しないかもしれない。 主役はあくまで新郎新婦ながらも、いざとなると緊張し、内容もあれこれ考えて、こっそりリハーサル……そんな人知れず頑張るスピーチ担当独女たちにも幸あれ（高山惠）
1,dokujo-tsushin,「5年前に別れた彼からメールが届いてビックリしました」とは尚美さん（36歳） 「彼の浮気が原因で別れたのですが、現在は独り身らしいことが書いてありました。ただ、私には婚約間近の恋人がいるのでスルー。もし自分に相手がいなかったら、復活愛はあったかも。出会いの機会が少ない独女にとって、いい時代と言えるのでは？」 彼と交際していた当時、尚美さんは実家に住んでいた。もしもメールも携帯電話もない時代だったら、恐らく彼からの連絡はあり得なかっただろう。 一方、美加子さん（38歳）は「一般電話だけの時代のほうが、縁を切りにくかったですよ」という。 「今はメモリ頼りになっている分、電話番号やメールアドレスを消去してしまえばそれまでってところがありますからね。ずるずる引きずろうと思えば引きずれるし、切ろうと思えば切れる。ひと昔前は彼の電話番号を暗記していたものです。受話器を上げたり戻したり……気持ちを断つのが難しかったなぁ」 知恵さん（34歳）も同意見だ。 「情報は自分が情報網を使えば入ってくるが 自分次第でシャットアウトできるもの。だけど私は、“別れた彼と絶縁すべき”とは思っていません。心が癒されるまでの期間は連絡を断ちきり、それ以降はメールを送るなどして、友達関係に戻ることが多いですね」 オール・オア・ナッシングだった以前に比べ、今は縁を切る、友人に戻る、メル友関係を続けるなど、別れた後の関係性を選択できるようになった。未練が残っているうちは辛いが、ツールを上手く使えばメリットを得られることもあるだろう。ただし、落とし穴もある。最後に、律子さん（35歳）のトホホな話を紹介。 「10年前につきあっていた元彼とは、節目節目にメールをしています。“マイミク”で日記にコメントを書き込んだりもするし、年賀状もやりとりする仲。元彼はすでに結婚して子どもが二人いるんですね。最近はすっかり中年太りしてきてマイホームパパって感じ。いい人ではあるけど、すっかり気持ちが冷めた今となっては、なぜ自分があれほど彼に執着していたのか疑問に思えてくる。過去の恋愛の想い出をきれい残したいのなら、知らないほうがいいこともあるかもしれません」 恋の思い出は脳内で美化されるもの。ネットなどで友情関係を続けるのはよいが、同時に淡く切ない恋の思い出は生活感、現実感にまみれてしまうことがあるので、繋ぎすぎには注意したいところ。ま、お互い様ですけどね。（来布十和）
2,dokujo-tsushin,今年に入ってから、“すっぴん”をブログで披露した芸能人は、小倉優子、安倍なつみ、モーニング娘。の田中れいな、優木まおみ、仲里依紗など、年齢も活躍しているジャンルも様々。私生活をリアルタイムに発信できるブログだからこそ、皆それぞれにリラックスした表情で自分のすっぴんを公開している。ファンにとっては、好きな芸能人の素顔が垣間見れる嬉しいサービスなのだろう。 では、なぜ芸能人のすっぴん顔披露に、彼女は疑問を抱いたのだろうか。「私のひがみだと言う事は重々承知なんですけど、“随分自分に自信があるんだな”って、素直にその美しさを認める事が出来なくて…」と話す香さん。「コメントに“すっぴんでもかわいい！”とか、“メイクしなくても全然OK！”とか賞賛ばかりが並ぶのを見越して、すっぴん写真を公開してるんだなって思うと何か複雑ですね」と付け加えた。 本来ファンサービスである為のすっぴん披露を、「話題を呼ぶ為や、コメントで褒められたいから」行っているのでは？ と、独女はつい“ナナメ”に見てしまう様だ。また、メーカーで営業をしている裕美さん（仮名／32歳）は「男性の“すっぴん幻想”には参りますね。そりゃ素顔がキレイならば私だってすっぴんで出社しますよ。でも、毎日少しでもキレイになりたいと一生懸命メイクしている努力も認めて欲しいな」と独女なりの乙女心を明かしてくれた。 「10代や20代前半の若い女の子がすっぴんを載せているのは、“ああ、やっぱり可愛いな”と心から思えるんですけど、私と同年代の人が披露していると自分のすっぴんと比べてゲンナリします」と、美しい芸能人すっぴんを見た後に独女達は人知れず傷ついているのである。 さて、芸能人のすっぴん披露、実際に男性の評判はどうなのだろうか。アパレル企業に勤める雄介さん（仮名／34歳）は「個人的に女性のメイクした顔に魅力を感じないから、すっぴんを見るとすごく可愛いと思う」と賛成派。一方、IT企業で働く徹さん（仮名／28歳）は「人によるけど、何でわざわざブログに載せるんだろうとは思います。本来、女性のすっぴんは大切な相手にだけ見せて欲しいんですよ。“あ、俺だけに見せてくれた”って感じで（笑）」と男性陣の意見もそれぞれの様だ。 今後、ブーム化する予感もあるブログでのすっぴん披露。肯定派と反対派に分かれている様だが、男性が女性の“すっぴん”に特別な想いを持っている事は確か。独女達も、いつかすっぴんを披露するその時の為に、素顔を磨く必要があるのかもしれない。
3,dokujo-tsushin,これは、４月に開催されたワコール人間科学研究所の記者発表「からだのエイジング（加齢による体型変化）について一定の法則を発見」での内容の一部。延べ4万人分の経年変化の数値を集計・分析したデータとともに、写真や映像で説明させるので説得力は抜群だ。 現実を直視させられた後に、体型変化の少ない人達の身体的特徴や日常の行動・意識を紹介。その主な内容は、日頃から体を動かし、姿勢をチェック、下着は必ず試着してフィット感を確かめるというもの。そして、パネルディスカッションでは、歩幅の広い歩き方を１年間続ける実験に参加した人が、背筋が伸び、脂肪が落ちたという結果などが紹介されていた。 興味は尽きず、知人たちに内容を伝えるとさまざまな意見や経験が聞けた。 「ずっと計測されているから、体型変化の少ない人はスポーツをしてたのでは？」という疑問もあったが、この回答は、「運動を一生懸命しているというより日常生活を気をつけている印象が強い。そして、ダイエットはあまりしたことがない」とのこと。 「叔母もそんなことを言っていた」と言うのはＹ子。60歳代の叔母さんが友人たちと温泉旅行に行ったとき、「バストの変化が少ないとほめられた」と喜んでいたので、バストのケア方法を尋ねたそうだ。「叔母はブラジャーを常に着

# 2. 自動機械学習 Automated Machine Learning
## 2.1 設定と制約条件

自動機械学習 Automated Machine Learning の設定と学習を行っていきます。

In [9]:
from azureml.automl.core.featurization import FeaturizationConfig
featurization_config = FeaturizationConfig()
# テキストデータの言語を指定します。日本語の場合は "jpn" と指定します。
featurization_config = FeaturizationConfig(dataset_language="jpn") # 英語の場合は下記をコメントアウトしてください。

In [10]:
# 明示的に `text` の列がテキストデータであると指定します。
featurization_config.add_column_purpose('text', 'Text')
featurization_config.blocked_transformers = ['TfIdf','CountVectorizer']  # BERT のみを利用したい場合はコメントアウトを外します

In [11]:
# 自動機械学習の設定
automl_settings = {
    "experiment_timeout_hours" : 0.25,  # 学習時間 (hour)
    "primary_metric": 'accuracy',  # 評価指標
    "max_concurrent_iterations": 4,  # 計算環境の最大並列数 
    "max_cores_per_iteration": -1,
    "enable_dnn": True, # 深層学習を有効
    "enable_early_stopping": False,
    "validation_size": 0.2,
    "verbosity": logging.INFO,
    "force_text_dnn": True,
    #"n_cross_validations": 5,
}

automl_config = AutoMLConfig(task = 'classification', 
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data=train_dataset,
                             label_column_name=target_column_name,
                             featurization=featurization_config,
                             path = project_folder,  # 必須パラメータ
                             **automl_settings
                            )

## 2.2 パイプラインの作成

In [12]:
from azureml.pipeline.steps import AutoMLStep
from azureml.pipeline.core import PipelineData, TrainingOutput


In [13]:
ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [14]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [15]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [16]:
pipeline.validate()
print("Pipeline validation complete")

Step automl_module is ready to be created [468bb8df]
Pipeline validation complete


## 2.3 パイプラインの実行

In [17]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [468bb8df][c4348728-7834-41f3-bfe8-5e611f9a79be], (This step will run and generate new outputs)
Submitted PipelineRun a171d8c0-c238-4e0c-a630-b182e86cb674
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/livedoor-news-classification-BERT-pipeline/runs/a171d8c0-c238-4e0c-a630-b182e86cb674?wsid=/subscriptions/9c0f91b8-eb2f-484c-979c-15848c098a6b/resourcegroups/azureml_text/workspaces/azureml_text


In [18]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …